In [7]:
def Save_Book(url1):
    import pandas as pd
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from bs4 import BeautifulSoup
    import time

    driver = webdriver.Chrome()
    driver.get(url1)
    time.sleep(2)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # 책 링크 추출
    book_links = soup.select('a.prod_info')
    hrefs = [link.get('href').split('/')[-1] for link in book_links]

    books_list = []

    for i in hrefs:
        url = f'https://product.kyobobook.co.kr/detail/{i}'
        driver.get(url)
        time.sleep(2)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 기본 정보 추출
        try:
            title_text = soup.select_one('#contents .prod_title_box h1 span').get_text(strip=True) if soup.select_one('#contents .prod_title_box h1 span') else None
            author_text = soup.select_one('#scrollSpyProdInfo .product_person .title_wrap h3 a span').get_text(strip=True) if soup.select_one('#scrollSpyProdInfo .product_person .title_wrap h3 a span') else None
            price_text = soup.select_one('.prod_price_box span.price span').get_text(strip=True) if soup.select_one('.prod_price_box span.price span') else None
            total_review_text = soup.select_one('.klover_review_box .box_top .caption span.val').get_text(strip=True) if soup.select_one('.klover_review_box .box_top .caption span.val') else None
            review_num_text = soup.select_one('.tab_wrap .tab_list_wrap .sps_inner li:nth-child(3) a span span').get_text(strip=True) if soup.select_one('.tab_wrap .tab_list_wrap .sps_inner li:nth-child(3) a span span') else None
            isbn_text = soup.select_one('.basic_info .tbl_row_wrap td').get_text(strip=True) if soup.select_one('.basic_info .tbl_row_wrap td') else None
            page_text = soup.select_one('.basic_info tr:nth-child(3) td').get_text(strip=True) if soup.select_one('.basic_info tr:nth-child(3) td') else None
            content_text = soup.select_one('.book_intro .intro_bottom div:nth-child(2)').get_text(strip=True) if soup.select_one('.book_intro .intro_bottom div:nth-child(2)') else None
            category = soup.select_one('#scrollSpyProdInfo > div.product_detail_area.book_intro > div.intro_book > ul > li').get_text(strip=True) if soup.select_one('#scrollSpyProdInfo > div.product_detail_area.book_intro > div.intro_book > ul > li') else None

            # 작가 정보 추출
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

                button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[8]/div[2]/div[3]/div/div[2]/button')
                driver.execute_script("arguments[0].click();", button_author)
                time.sleep(2)
                author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.book_publish_review')
                author_info_text = author_info_section.text

            except Exception as e:
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)

                    button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/div[3]/div[2]/div[2]/button')
                    driver.execute_script("arguments[0].click();", button_author)
                    time.sleep(2)
                    author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.product_person > div:nth-child(2) > div.writer_info_box > div.auto_overflow_wrap.overflow.active > div.auto_overflow_contents > div > p')
                    author_info_text = author_info_section.text
                except Exception as e:
                    try:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(2)

                        button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[3]/div[2]/div[2]/button')

                        driver.execute_script("arguments[0].click();", button_author)
                        time.sleep(2)
                        author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.product_person > div.round_gray_box > div.writer_info_box > div.auto_overflow_wrap.overflow.active > div.auto_overflow_contents > div > p')
                        author_info_text = author_info_section.text
                    except Exception as e:
                        try:
                            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            time.sleep(2)

                            button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[3]/div/div[2]/button')

                            driver.execute_script("arguments[0].click();", button_author)
                            time.sleep(2)
                            author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.product_person > div:nth-child(2) > div.writer_info_box > div > div.auto_overflow_contents > div > p')
                            author_info_text = author_info_section.text
                        except Exception as e:
                            try:
                                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                                time.sleep(2)

                                button_author = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[8]/div[2]/div[3]/div[2]/div[2]/button')

                                driver.execute_script("arguments[0].click();", button_author)
                                time.sleep(2)
                                author_info_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.product_person > div:nth-child(2) > div.writer_info_box > div.auto_overflow_wrap.overflow.active > div.auto_overflow_contents > div > p')
                                author_info_text = author_info_section.text
                            except Exception as e:
                                author_info_text = "작가 정보를 불러오는데 실패하였습니다."


            # 서평 추출
            try:
                button_review = driver.find_element(By.XPATH, '//*[@id="scrollSpyProdInfo"]/div[11]/div[2]/div[2]/button')
                driver.execute_script("arguments[0].click();", button_review)
                time.sleep(2)
                review_section = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.book_publish_review')
                review_text = review_section.text

            except Exception as e:
                review_text = "서평 정보를 불러오는데 실패하였습니다."


            books_list.append({
                
                'title': title_text,
                'author': author_text,
                'price': price_text,
                'rev_avg': total_review_text,
                'rev_cou': review_num_text,
                'ISBN': isbn_text,
                'page': page_text,
                'content': content_text,
                'autho_info': author_info_text,
                'book_rev': review_text,
                'category': category})


        except Exception as e:
            print(f"책 정보를 가져오는 도중 오류 발생: {e}")

    df = pd.DataFrame(books_list)
    df['saleCmdtid']=hrefs
    return df


In [8]:
import pandas as pd

all_books_df = pd.DataFrame()

# 연도별로 반복
for year in range(19, 24): 
    for i in range(1, 11):
        url = f'https://product.kyobobook.co.kr/bestseller/total?period=004#?page={i}&per=20&period=004&ymw=20{year}&bsslBksClstCode=A'
        current_page_df = Save_Book(url)
        all_books_df = pd.concat([all_books_df, current_page_df], ignore_index=True)